In [160]:
import pandas as pd
import numpy as np
from Functions import Cleaning_Functions
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics, ensemble

fun = Cleaning_Functions()

clean = pd.read_csv("../data/clean.csv")
clean  = clean.drop("continent", axis =1)

In [161]:
def standarize_data(df):
        """
        Input: a dataset
        action: returns numeric column values scaled by mean and standard deviation
        """
        numeric_data = df.select_dtypes(include=['float64', 'int64'])
        for i in numeric_data.columns:
            df[i] = (df[i] - df[i].mean())/df[i].std()
        return df

In [162]:
market_data = fun.delete_id_columns(clean) #1
market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
market_data = fun.replace_NAN_with_na(market_data) #3
market_data = fun.entry_to_lowercase(market_data) #4
market_data = fun.remove_underscores_spaces(market_data) #5
market_data = fun.convert_to_categorical(market_data) #6
market_data = fun.impute_data(market_data)
market_data = standarize_data(market_data)


In [163]:
#Ana's fuc
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [164]:
X, y = get_dummyXs_y(market_data, "Market_Orientation")
X_tr, X_te, y_tr, y_te = model_selection.train_test_split(X,y, test_size = 0.3, random_state = 50)


In [165]:
def fit_predict(clf, X_tr, X_te, y_tr, y_te):
    clf.fit(X_tr,y_tr)
    pred = clf.predict(X_te)
    mse = metrics.mean_squared_error(y_te, pred)
    
    return "MSE: {} ".format(mse)



def tune_parameters(X_train, y_train, clf, param_dict, cv=5):
    
   
    
    best_model = model_selection.GridSearchCV(clf, param_dict, cv=cv, scoring = "neg_mean_squared_error", n_jobs =-1, verbose=3)
    
    best_model.fit(X_train, y_train)
    
    print("Best Parameters: {} \n Training MSE: {} \n Parameter Index: {}".format(best_model.best_params_,best_model.best_score_,best_model.best_index_) ) # best is alpha = 0


    #uses gridsearch, prints best parameters, best model, its MSE on the training set
    #returns classifer
    
    return clf

test_mse_market = []

In [166]:
lasso_cv=linear_model.LassoCV(alphas=np.arange(0.01,2,0.001))
model_cv=lasso_cv.fit(X_tr,y_tr)
model_cv.alpha_

0.01

In [167]:
best_lassom = linear_model.Lasso(alpha = model_cv.alpha_)
best_lassom.fit(X_tr,y_tr)
y_pred_lasso = best_lassom.predict(X_te)
MSE_lasso_market = metrics.mean_squared_error(y_te,y_pred_lasso)
MSE_lasso_market

0.6794566845283756

In [176]:
feature_imp_mark  = pd.DataFrame([X_tr.columns, best_lassom.coef_]).T
feature_imp_mark = feature_imp_mark.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_mark.iloc[0:15,:]

,0,1
0,Country_burkinafaso,0.412779
1,crop_harvest_1_na,0.11309
2,Country_vietnam,0.084943
3,crop_intercrop_1_na,0.057034
4,crop_name_1_lemons,0.0139548
5,LandCultivated,0.00674036
6,livestock_prodsales_USD_PPP_pHH_Yr,0.00408668
7,Gender_FemaleControl,6.43889e-18
8,Country_cambodia,0
9,Country_elsalvador,0


In [169]:
clean = pd.read_csv("../data/clean.csv")
clean  = clean.drop("continent", axis =1)
ppi_data = fun.delete_id_columns(clean) #1
ppi_data, pred_ppi = fun.drop_response_rows_with_NAs(ppi_data, "PPI_Likelihood", "Market_Orientation") #2
ppi_data = fun.replace_NAN_with_na(ppi_data) #3
ppi_data = fun.entry_to_lowercase(ppi_data) #4
ppi_data = fun.remove_underscores_spaces(ppi_data) #5
ppi_data = fun.convert_to_categorical(ppi_data) #6
ppi_data = fun.impute_data(ppi_data)
ppi_data = standarize_data(ppi_data)

In [170]:
X, y = get_dummyXs_y(ppi_data, "PPI_Likelihood")
X_tr, X_te, y_tr, y_te = model_selection.train_test_split(X,y, test_size = 0.3, random_state = 50)

lasso_cv=linear_model.LassoCV(alphas=np.arange(0.01,2,0.001))
model_cv=lasso_cv.fit(X_tr,y_tr)
model_cv.alpha_

0.01

In [171]:
best_lassop = linear_model.Lasso(alpha = model_cv.alpha_)
best_lassop.fit(X_tr,y_tr)
y_pred_lasso = best_lassop.predict(X_te)
MSE_lasso_ppi = metrics.mean_squared_error(y_te,y_pred_lasso)
MSE_lasso_ppi

0.526585014110179

In [180]:
feature_imp_ppi  = pd.DataFrame([X_tr.columns, best_lassop.coef_],  ).T
feature_imp_ppi = feature_imp_ppi.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_ppi.iloc[0:15,:]

,0,1
0,Country_mali,1.59999
1,Country_zambia,1.57829
2,Country_malawi,0.72713
3,Country_tanzania,0.239393
4,Head_EducationLevel_noschool,0.20873
5,HFIAS_status_severelyfi,0.175241
6,Country_burkinafaso,0.110341
7,NrofMonthsFoodInsecure,0.0969462
8,HHsizemembers,0.0737167
9,HouseholdType_couple,0.0723588


In [187]:
combined_imp = pd.concat([feature_imp_ppi, feature_imp_mark], axis = 1).iloc[0:20,:]
combined_imp.columns = ["PPI Likelihood","Abs Coefficent","Market Orientation","Abs Coefficent"]
combined_imp

,PPI Likelihood,Abs Coefficent,Market Orientation,Abs Coefficent
0,Country_mali,1.59999,Country_burkinafaso,0.412779
1,Country_zambia,1.57829,crop_harvest_1_na,0.11309
2,Country_malawi,0.72713,Country_vietnam,0.084943
3,Country_tanzania,0.239393,crop_intercrop_1_na,0.057034
4,Head_EducationLevel_noschool,0.20873,crop_name_1_lemons,0.0139548
5,HFIAS_status_severelyfi,0.175241,LandCultivated,0.00674036
6,Country_burkinafaso,0.110341,livestock_prodsales_USD_PPP_pHH_Yr,0.00408668
7,NrofMonthsFoodInsecure,0.0969462,Gender_FemaleControl,6.43889e-18
8,HHsizemembers,0.0737167,Country_cambodia,0
9,HouseholdType_couple,0.0723588,Country_elsalvador,0
